In [1]:
import numpy as np
import os
import denoise_model
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


### Data preprocessing (MNIST)

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
image_size = x_train.shape[1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])

In [3]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=8,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)
datagen.fit(x_train)

### Train validation model

In [4]:
nb_epochs = 200
batch_size = 256

In [5]:
def train_val_model(model, denoise_model, denoise_model_path):
    if not os.path.exists(denoise_model_path):
        test_noise = np.random.normal(loc=0, scale=0.1, size=x_test.shape)
        x_test_noise = np.clip(x_test + test_noise, 0, 1)
        min_loss = np.finfo(np.float32).max
        patience = 0
        max_patience = 30
        for e in range(nb_epochs):
            batches = 0
            for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
                train_noise = np.random.normal(loc=0, scale=0.1, size=x_batch.shape)
                x_batch_noise = np.clip(x_batch + train_noise, 0, 1)
                denoise_model.fit(x_batch_noise, y_batch, verbose=0)
                model.fit(x_batch, y_batch, verbose=0)
                batches += 1
                if batches >= len(x_train) // batch_size: break
            denoise_model_loss = denoise_model.evaluate(x_test_noise, y_test)
            orig_model_loss = model.evaluate(x_test, y_test)
            loss = denoise_model_loss + orig_model_loss
            print('epoch ', e, ', loss: ', loss)
            if loss < min_loss:
                min_loss = loss
                denoise_model.save_weights(denoise_model_path)
                patience = 0
            else: patience += 1
            if patience > max_patience:
                break

In [6]:
def train_val_model_ver2(model, denoise_model, denoise_model_path):
    if not os.path.exists(denoise_model_path):
        train_noise = np.random.normal(loc=0, scale=0.1, size=x_train.shape)
        test_noise = np.random.normal(loc=0, scale=0.1, size=x_test.shape)
        x_train_noise = np.clip(x_train + train_noise, 0, 1)
        x_test_noise = np.clip(x_test + test_noise, 0, 1)
        callbacks = [EarlyStopping(monitor='val_accuracy', patience=30), ModelCheckpoint(denoise_model_path, monitor='val_accuracy', save_best_only=True)]
        denoise_model.fit(x_train_noise, y_train, epochs=nb_epochs, batch_size=batch_size, validation_data=(x_test_noise, y_test), callbacks=callbacks)

In [7]:
dm_gaussian_blur_path = 'model/dm_gaussian_blur.h5'
dm_median_blur_path = 'model/dm_median_blur.h5'
dm_nl_means_path = 'model/dm_nl_means.h5'
dm_dae_path = 'model/dm_dae.h5'
dae_path = 'model/DAE.h5'

dm_gaussian_blur = denoise_model.DM_GAUSSIAN_BLUR()
dm_median_blur = denoise_model.DM_MEDIAN_BLUR()
dm_nl_means = denoise_model.DM_NL_MEANS()
dm_dae = denoise_model.DM_DAE()

In [8]:
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

dm_dae.denoiser.trainable = False
dm_dae.denoiser.compile(loss='mse', optimizer='adam')

dm_gaussian_blur.model.compile(loss=loss_object, optimizer='adam')
dm_median_blur.model.compile(loss=loss_object, optimizer='adam')
dm_nl_means.model.compile(loss=loss_object, optimizer='adam')
dm_dae.model.compile(loss=loss_object, optimizer='adam')

dm_gaussian_blur.denoise_model.compile(loss=loss_object, optimizer='adam', metrics=['accuracy'])
dm_median_blur.denoise_model.compile(loss=loss_object, optimizer='adam', metrics=['accuracy'])
dm_nl_means.denoise_model.compile(loss=loss_object, optimizer='adam', metrics=['accuracy'])
dm_dae.denoise_model.compile(loss=loss_object, optimizer='adam', metrics=['accuracy'])

dm_dae.denoiser.load_weights(dae_path)

In [9]:
train_val_model_ver2(dm_gaussian_blur.model, dm_gaussian_blur.denoise_model, dm_gaussian_blur_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 8s 138us/step - loss: 0.3614 - accuracy: 0.8895 - val_loss: 0.0773 - val_accuracy: 0.9767
Epoch 2/200
60000/60000 [==============================] - 7s 118us/step - loss: 0.1252 - accuracy: 0.9631 - val_loss: 0.0529 - val_accuracy: 0.9828
Epoch 3/200
60000/60000 [==============================] - 7s 122us/step - loss: 0.0904 - accuracy: 0.9727 - val_loss: 0.0432 - val_accuracy: 0.9862
Epoch 4/200
60000/60000 [==============================] - 7s 121us/step - loss: 0.0758 - accuracy: 0.9770 - val_loss: 0.0402 - val_accuracy: 0.9873
Epoch 5/200
60000/60000 [==============================] - 7s 121us/step - loss: 0.0644 - accuracy: 0.9806 - val_loss: 0.0356 - val_accuracy: 0.9883
Epoch 6/200
60000/60000 [==============================] - 7s 121us/step - loss: 0.0555 - accuracy: 0.9835 - val_loss: 0.0330 - val_accuracy: 0.9891
Epoch 7/200
60000/60000 [==============================]

60000/60000 [==============================] - 7s 120us/step - loss: 0.0080 - accuracy: 0.9973 - val_loss: 0.0371 - val_accuracy: 0.9927
Epoch 56/200
60000/60000 [==============================] - 7s 119us/step - loss: 0.0074 - accuracy: 0.9973 - val_loss: 0.0362 - val_accuracy: 0.9927
Epoch 57/200
60000/60000 [==============================] - 7s 121us/step - loss: 0.0068 - accuracy: 0.9977 - val_loss: 0.0387 - val_accuracy: 0.9932
Epoch 58/200
60000/60000 [==============================] - 7s 120us/step - loss: 0.0071 - accuracy: 0.9975 - val_loss: 0.0359 - val_accuracy: 0.9930
Epoch 59/200
60000/60000 [==============================] - 7s 120us/step - loss: 0.0073 - accuracy: 0.9974 - val_loss: 0.0420 - val_accuracy: 0.9910
Epoch 60/200
60000/60000 [==============================] - 7s 120us/step - loss: 0.0084 - accuracy: 0.9970 - val_loss: 0.0396 - val_accuracy: 0.9924
Epoch 61/200
60000/60000 [==============================] - 7s 120us/step - loss: 0.0078 - accuracy: 0.9972 - val

In [10]:
train_val_model_ver2(dm_median_blur.model, dm_median_blur.denoise_model, dm_median_blur_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 20s 336us/step - loss: 0.3328 - accuracy: 0.8975 - val_loss: 0.0788 - val_accuracy: 0.9762
Epoch 2/200
60000/60000 [==============================] - 20s 333us/step - loss: 0.1204 - accuracy: 0.9635 - val_loss: 0.0539 - val_accuracy: 0.9823
Epoch 3/200
60000/60000 [==============================] - 20s 336us/step - loss: 0.0906 - accuracy: 0.9728 - val_loss: 0.0450 - val_accuracy: 0.9847
Epoch 4/200
60000/60000 [==============================] - 20s 332us/step - loss: 0.0745 - accuracy: 0.9772 - val_loss: 0.0443 - val_accuracy: 0.9858
Epoch 5/200
60000/60000 [==============================] - 20s 335us/step - loss: 0.0631 - accuracy: 0.9801 - val_loss: 0.0379 - val_accuracy: 0.9875
Epoch 6/200
60000/60000 [==============================] - 20s 339us/step - loss: 0.0563 - accuracy: 0.9829 - val_loss: 0.0349 - val_accuracy: 0.9877
Epoch 7/200
60000/60000 [=========================

Epoch 55/200
60000/60000 [==============================] - 20s 338us/step - loss: 0.0078 - accuracy: 0.9973 - val_loss: 0.0516 - val_accuracy: 0.9905
Epoch 56/200
60000/60000 [==============================] - 20s 332us/step - loss: 0.0072 - accuracy: 0.9975 - val_loss: 0.0485 - val_accuracy: 0.9918
Epoch 57/200
60000/60000 [==============================] - 20s 329us/step - loss: 0.0074 - accuracy: 0.9976 - val_loss: 0.0455 - val_accuracy: 0.9913
Epoch 58/200
60000/60000 [==============================] - 20s 335us/step - loss: 0.0084 - accuracy: 0.9972 - val_loss: 0.0485 - val_accuracy: 0.9900
Epoch 59/200
60000/60000 [==============================] - 20s 338us/step - loss: 0.0069 - accuracy: 0.9976 - val_loss: 0.0502 - val_accuracy: 0.9909
Epoch 60/200
60000/60000 [==============================] - 20s 338us/step - loss: 0.0086 - accuracy: 0.9971 - val_loss: 0.0499 - val_accuracy: 0.9907
Epoch 61/200
60000/60000 [==============================] - 20s 339us/step - loss: 0.0073 - ac

In [11]:
train_val_model_ver2(dm_dae.model, dm_dae.denoise_model, dm_dae_path)

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 4s 69us/step - loss: 0.3160 - accuracy: 0.9029 - val_loss: 0.1008 - val_accuracy: 0.9689
Epoch 2/200
60000/60000 [==============================] - 4s 62us/step - loss: 0.1262 - accuracy: 0.9638 - val_loss: 0.0807 - val_accuracy: 0.9738
Epoch 3/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.1002 - accuracy: 0.9703 - val_loss: 0.0723 - val_accuracy: 0.9779
Epoch 4/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0874 - accuracy: 0.9743 - val_loss: 0.0712 - val_accuracy: 0.9789
Epoch 5/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.0765 - accuracy: 0.9772 - val_loss: 0.0680 - val_accuracy: 0.9807
Epoch 6/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0690 - accuracy: 0.9793 - val_loss: 0.0704 - val_accuracy: 0.9813
Epoch 7/200
60000/60000 [==============================] - 4s 

Epoch 56/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0107 - accuracy: 0.9962 - val_loss: 0.1128 - val_accuracy: 0.9836
Epoch 57/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0106 - accuracy: 0.9961 - val_loss: 0.1093 - val_accuracy: 0.9836
Epoch 58/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.0102 - accuracy: 0.9967 - val_loss: 0.1135 - val_accuracy: 0.9828
Epoch 59/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0100 - accuracy: 0.9965 - val_loss: 0.1233 - val_accuracy: 0.9837
Epoch 60/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0088 - accuracy: 0.9968 - val_loss: 0.1193 - val_accuracy: 0.9841
Epoch 61/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0115 - accuracy: 0.9960 - val_loss: 0.1097 - val_accuracy: 0.9837
Epoch 62/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.0096 - accuracy: 0.9970